In [1]:
import sys

!{sys.executable} predict.py \
    -i ./example \
    -f mol2 \
    -o ./output/ \
    -l ./example.log

Start data preparation: example
Start prediction: example
example prediction: 100%|█████████████████████████| 3/3 [00:25<00:00,  8.35s/it]
==> Finished processing example, Failed samples: 0



In [2]:
import py3Dmol
import os
import glob

def read_pdb_as_string(file_path):
    with open(file_path, 'r') as file:
        return file.read()

def visualize_molecules_sequential(index=0, base_input='./example', base_output='./output/example'):
    directories = sorted([d for d in os.listdir(base_input) if os.path.isdir(os.path.join(base_input, d))])
    if not directories:
        print("No valid input directories found.")
        return

    if index >= len(directories):
        print(f"Index {index} out of range. Only {len(directories)} directories available.")
        return

    selected_dir = directories[index]
    ligand_path = os.path.join(base_input, selected_dir, 'ligand.mol2')
    protein_path = os.path.join(base_input, selected_dir, 'protein.pdb')
    pocket_paths = glob.glob(os.path.join(base_output, selected_dir, 'grid*.mol2'))

    if not pocket_paths:
        print(f"No predicted pockets found for: {selected_dir}")
        return

    try:
        view = py3Dmol.view(width=1000, height=800)
        protein_data = read_pdb_as_string(protein_path)
        ligand_data = read_pdb_as_string(ligand_path)

        if protein_data:
            view.addModel(protein_data, 'pdb')
            view.setStyle({'model': 0}, {'cartoon': {'color': 'white'}})
        if ligand_data:
            view.addModel(ligand_data, 'mol2')
            view.setStyle({'model': 1}, {'stick': {'radius': 0.3, 'color': 'red'}})

        for i, pocket_path in enumerate(pocket_paths):
            pocket_data = read_pdb_as_string(pocket_path)
            if pocket_data:
                view.addModel(pocket_data, 'mol2')
                view.setStyle({'model': 2 + i}, {'sphere': {'opacity': 0.5, 'color': 'blue'}})

        view.zoomTo()
        print(f"Showing: {selected_dir}")
        return view
    except Exception as e:
        print(f"An error occurred while visualizing {selected_dir}: {e}")


In [3]:
view = visualize_molecules_sequential(index=0)
if view:
    view.show()


Showing: 1tjw_A


3Dmol.js failed to load for some reason. Please check your browser console for error messages.